In [6]:
from ruptura.RupturaTable import RupturaTable
rupturaTable = RupturaTable()
files = ['ruptura-2018-dezembro-1oQ.csv', 'ruptura-2018-dezembro-2oQ.csv', 'ruptura-2019-fevereiro.csv', 'ruptura-2019-janeiro-1oQ.csv', 'ruptura-2019-janeiro-2oQ.csv', 'ruptura-2019-marco.csv']
clipData = ['01/11/2018', '01/04/2019']
rupturaTable.generate(files,clipData)

In [9]:
from ruptura.CreateBatch import CreateBatch
createBatch = CreateBatch()
amostras = createBatch.create(rupturaTable.getData(), 'BISCOITO BARBIERI PAPA OVO 150G')
createBatch.exportBatch(amostras)

In [1]:
from ruptura.CreateBatch import CreateBatch
createBatch = CreateBatch()
amostras = createBatch.loadBatch()
X, Y, Ytest = createBatch.batch(amostras)

In [31]:
data = rupturaTable.getData()
produtos = list(collections.Counter(data.loc[:,'Produto']))
todosBiscoitos  = []
for prod in produtos:
    if 'BISCOITO BARBIERI' in prod:
        todosBiscoitos.append(prod)
todasAmostras = {}
data = rupturaTable.getData()
for nomeDoProduto in todosBiscoitos:
    amostras = createBatch.create(data, nomeDoProduto)
    todasAmostras.update(dict(amostras))
            

In [39]:
createBatch.exportBatch(todasAmostras)

In [9]:
data.columns

Index(['Data Hora de encerramento da OS', 'C digo do cliente',
       'Nome do cliente', 'Categoria do cliente', 'Filial', 'Produto',
       'Ocorr ncia', 'Ocorr ncia2', 'Rede Visitada', 'Itera es', 'Unnamed 10',
       'dataDeOcorrencia'],
      dtype='object')

In [14]:
data = rupturaTable.getData()

In [16]:
dataMart = data[data.loc[:,'Nome do cliente'].values == 'Mart Minas Paracatu - Loja 224'].copy()

In [18]:
dataMart[dataMart.loc[:,'Produto'].values == 'BISCOITO BARBIERI ESCALDADO 150G']

,Data Hora de encerramento da OS,C digo do cliente,Nome do cliente,Categoria do cliente,Filial,Produto,Ocorr ncia,Ocorr ncia2,Rede Visitada,Itera es,Unnamed 10,dataDeOcorrencia
24,01/12/2018 09:22:01,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-110
236104,08/12/2018 11:24:48,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-103
358889,12/12/2018 11:45:16,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-99
380287,13/12/2018 16:30:08,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Presen a,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-98
629292,20/12/2018 16:31:04,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-91
692809,23/12/2018 10:04:43,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Presen a,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-88
748011,23/12/2018 10:05:48,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Presen a,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-88
832741,27/12/2018 17:47:14,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-84
900138,29/12/2018 15:39:10,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Reposi o,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-82
1000731,02/02/2019 09:57:35,MARTMINAS-20,Mart Minas Paracatu - Loja 224,Canal Alimentar,BARBIERI,BISCOITO BARBIERI ESCALDADO 150G,Presen a,Sem Unidades Estocadas Armazenadas,NaN,Produto:Ocorrência2:1,NaN,-47


In [3]:
amostras['BISCOITO BARBIERI ESCALDADO 150GMart Minas Paracatu Loja 224']

[[0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1],
 [1, 0, 0,